In [1]:
import pandas as pd

In [7]:
# Read the Excel file into a DataFrame
df = pd.read_excel("cleaned_data_indigenous_list.xlsx")
# Remove rows with missing values in the "web-site" column
df.dropna(subset=['Band Website'], inplace=True)
# Reset the index
df.reset_index(drop=True, inplace=True)
df


,Band Name,Band Number,Band Address,Band Phone,Band Fax,Band Website
0,Abegweit,1,"PO Box 36, 11222 St. Peters Road, Scotchfort, PE",(902) 676-2353,(902) 676-3206,https://abegweit.ca/
1,Lennox Island,2,"2 Eagle Feather Trail, Lennox Island, PE",(902) 831-2779,(902) 831-3153,https://lennoxisland.com/
2,Elsipogtog First Nation,3,"373 Big Cove Road, Elsipogtog First Nation, NB",(506) 524-8654,(506) 523-8230,http://www.elsipogtog.ca/
3,Buctouche MicMac,4,"9 Reserve Road, Buctouche Reserve, NB",(506) 743-2520,(506) 743-8995,http://www.nsmdc.ca/main.html
4,Madawaska Maliseet First Nation,6,"1771 Principlae Street, Madawaska Maliseet Fir...",(506) 739-9765,(506) 735-0024,http://www.madawaskamaliseetfirstnation.com/
...,...,...,...,...,...,...
483,Ka'a'gee Tu First Nation,768,"PO BOX 4428, HAY RIVER, NT",(867) 825-2000,(867) 825-2002,http://www.dehcho.org/members/kakisa.htm
484,Jean Marie River First Nation,770,"GENERAL DELIVERY, JEAN MARIE RIVER, NT",(867) 809-2000,(867) 809-2002,http://www.jmrfn.com
485,West Point First Nation,772,"1 - 47031 MACKENZIE HWY, HAY RIVER, NT",(867) 874-6677,(867) 874-2486,http://www.dehcho.org/members/west_point.htm
486,Dechi Laot'i First Nations,774,"PO BOX 34, WEKWETI, NT",(867) 713-2511,(867) 713-2064,http://www.wekweti.ca/


Emails scraper

In [17]:
from splinter import Browser
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.chrome.options import Options
import re
import time
import pandas as pd

# Set up the browser (you need to have ChromeDriver installed and in your PATH)
executable_path = '/opt/homebrew/bin/chromedriver'  # Replace with the actual path to your chromedriver executable
chrome_options = Options()
chrome_options.add_argument('--headless')  # Run in headless mode (no GUI)

# Set the maximum time to wait for a website to respond (in seconds)
max_wait_time = 30

# Assuming df is your DataFrame containing the 'Band Website' column
websites = df['Band Website']

# Iterate through the websites and extract emails
for i, url in enumerate(websites):
    try:
        # Visit the website with a maximum wait time
        browser = Browser('chrome', executable_path=executable_path, options=chrome_options)
        browser.visit(url)
        start_time = time.time()

        # Wait until the website is loaded or until the maximum wait time is reached
        while (time.time() - start_time) < max_wait_time:
            if browser.is_element_present_by_css('body'):
                break

        # Get the page content if the website loaded successfully
        if (time.time() - start_time) < max_wait_time:
            page_content = browser.html

            # Define a regular expression to extract email addresses
            email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'

            # Extract email addresses from the page content
            emails = set(re.findall(email_pattern, page_content))

            # Create column names based on the index (i.e., email1, email2)
            email_columns = [f"email{i + 1}" for i in range(len(emails))]

            # If there are emails, create a dictionary to store the emails with appropriate column names
            if emails:
                email_data = {col: email for col, email in zip(email_columns, emails)}

                # Add the email data to the DataFrame for the corresponding row
                df.loc[i, email_columns] = pd.Series(email_data, dtype='object')  # Specify dtype='object'

    except WebDriverException as e:
        print(f"An error occurred while visiting {url}: {str(e)}")

    finally:
        browser.quit()


An error occurred while visiting http://www.gespeg.ca/english.html: Message: unknown error: net::ERR_CONNECTION_TIMED_OUT
  (Session info: headless chrome=117.0.5938.149)
Stacktrace:
0   chromedriver                        0x000000010337265c chromedriver + 4318812
1   chromedriver                        0x000000010336ad00 chromedriver + 4287744
2   chromedriver                        0x0000000102f9c7ec chromedriver + 296940
3   chromedriver                        0x0000000102f9567c chromedriver + 267900
4   chromedriver                        0x0000000102f849a4 chromedriver + 199076
5   chromedriver                        0x0000000102f85ef8 chromedriver + 204536
6   chromedriver                        0x0000000102f84cd0 chromedriver + 199888
7   chromedriver                        0x0000000102f83b48 chromedriver + 195400
8   chromedriver                        0x0000000102f83af4 chromedriver + 195316
9   chromedriver                        0x0000000102f81fa4 chromedriver + 188324
10  c

An error occurred while visiting http://www.bgfn.on.ca/: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: headless chrome=117.0.5938.149)
Stacktrace:
0   chromedriver                        0x000000010455665c chromedriver + 4318812
1   chromedriver                        0x000000010454ed00 chromedriver + 4287744
2   chromedriver                        0x00000001041807ec chromedriver + 296940
3   chromedriver                        0x000000010417967c chromedriver + 267900
4   chromedriver                        0x00000001041689a4 chromedriver + 199076
5   chromedriver                        0x0000000104169ef8 chromedriver + 204536
6   chromedriver                        0x0000000104168cd0 chromedriver + 199888
7   chromedriver                        0x0000000104167b48 chromedriver + 195400
8   chromedriver                        0x0000000104167af4 chromedriver + 195316
9   chromedriver                        0x0000000104165fa4 chromedriver + 188324
10  chromedriver   

An error occurred while visiting http://www.win-tlua.ca/: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: headless chrome=117.0.5938.149)
Stacktrace:
0   chromedriver                        0x00000001009b265c chromedriver + 4318812
1   chromedriver                        0x00000001009aad00 chromedriver + 4287744
2   chromedriver                        0x00000001005dc7ec chromedriver + 296940
3   chromedriver                        0x00000001005d567c chromedriver + 267900
4   chromedriver                        0x00000001005c49a4 chromedriver + 199076
5   chromedriver                        0x00000001005c5ef8 chromedriver + 204536
6   chromedriver                        0x00000001005c4cd0 chromedriver + 199888
7   chromedriver                        0x00000001005c3b48 chromedriver + 195400
8   chromedriver                        0x00000001005c3af4 chromedriver + 195316
9   chromedriver                        0x00000001005c1fa4 chromedriver + 188324
10  chromedriver  

An error occurred while visiting http://www.eabametoong.firstnation.ca/: Message: unknown error: net::ERR_CONNECTION_TIMED_OUT
  (Session info: headless chrome=117.0.5938.149)
Stacktrace:
0   chromedriver                        0x000000010261e65c chromedriver + 4318812
1   chromedriver                        0x0000000102616d00 chromedriver + 4287744
2   chromedriver                        0x00000001022487ec chromedriver + 296940
3   chromedriver                        0x000000010224167c chromedriver + 267900
4   chromedriver                        0x00000001022309a4 chromedriver + 199076
5   chromedriver                        0x0000000102231ef8 chromedriver + 204536
6   chromedriver                        0x0000000102230cd0 chromedriver + 199888
7   chromedriver                        0x000000010222fb48 chromedriver + 195400
8   chromedriver                        0x000000010222faf4 chromedriver + 195316
9   chromedriver                        0x000000010222dfa4 chromedriver + 188324


An error occurred while visiting http://www.wapekeka.ca/: Message: unknown error: net::ERR_CONNECTION_TIMED_OUT
  (Session info: headless chrome=117.0.5938.149)
Stacktrace:
0   chromedriver                        0x0000000104c8e65c chromedriver + 4318812
1   chromedriver                        0x0000000104c86d00 chromedriver + 4287744
2   chromedriver                        0x00000001048b87ec chromedriver + 296940
3   chromedriver                        0x00000001048b167c chromedriver + 267900
4   chromedriver                        0x00000001048a09a4 chromedriver + 199076
5   chromedriver                        0x00000001048a1ef8 chromedriver + 204536
6   chromedriver                        0x00000001048a0cd0 chromedriver + 199888
7   chromedriver                        0x000000010489fb48 chromedriver + 195400
8   chromedriver                        0x000000010489faf4 chromedriver + 195316
9   chromedriver                        0x000000010489dfa4 chromedriver + 188324
10  chromedrive

...

An error occurred while visiting http://www.horselakefirstnation.com/: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: headless chrome=117.0.5938.149)
Stacktrace:
0   chromedriver                        0x0000000102a4265c chromedriver + 4318812
1   chromedriver                        0x0000000102a3ad00 chromedriver + 4287744
2   chromedriver                        0x000000010266c7ec chromedriver + 296940
3   chromedriver                        0x000000010266567c chromedriver + 267900
4   chromedriver                        0x00000001026549a4 chromedriver + 199076
5   chromedriver                        0x0000000102655ef8 chromedriver + 204536
6   chromedriver                        0x0000000102654cd0 chromedriver + 199888
7   chromedriver                        0x0000000102653b48 chromedriver + 195400
8   chromedriver                        0x0000000102653af4 chromedriver + 195316
9   chromedriver                        0x0000000102651fa4 chromedriver + 188324
10  c

An error occurred while visiting http://www.wrfn.yk.ca/: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: headless chrome=117.0.5938.149)
Stacktrace:
0   chromedriver                        0x00000001052be65c chromedriver + 4318812
1   chromedriver                        0x00000001052b6d00 chromedriver + 4287744
2   chromedriver                        0x0000000104ee87ec chromedriver + 296940
3   chromedriver                        0x0000000104ee167c chromedriver + 267900
4   chromedriver                        0x0000000104ed09a4 chromedriver + 199076
5   chromedriver                        0x0000000104ed1ef8 chromedriver + 204536
6   chromedriver                        0x0000000104ed0cd0 chromedriver + 199888
7   chromedriver                        0x0000000104ecfb48 chromedriver + 195400
8   chromedriver                        0x0000000104ecfaf4 chromedriver + 195316
9   chromedriver                        0x0000000104ecdfa4 chromedriver + 188324
10  chromedriver   

An error occurred while visiting http://www.burnslakeband.ca/: Message: unknown error: net::ERR_CONNECTION_REFUSED
  (Session info: headless chrome=117.0.5938.149)
Stacktrace:
0   chromedriver                        0x000000010468265c chromedriver + 4318812
1   chromedriver                        0x000000010467ad00 chromedriver + 4287744
2   chromedriver                        0x00000001042ac7ec chromedriver + 296940
3   chromedriver                        0x00000001042a567c chromedriver + 267900
4   chromedriver                        0x00000001042949a4 chromedriver + 199076
5   chromedriver                        0x0000000104295ef8 chromedriver + 204536
6   chromedriver                        0x0000000104294cd0 chromedriver + 199888
7   chromedriver                        0x0000000104293b48 chromedriver + 195400
8   chromedriver                        0x0000000104293af4 chromedriver + 195316
9   chromedriver                        0x0000000104291fa4 chromedriver + 188324
10  chromedr

An error occurred while visiting http://www.tetlitgwichin.ca/AboutTetlitGwichin: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: headless chrome=117.0.5938.149)
Stacktrace:
0   chromedriver                        0x00000001006da65c chromedriver + 4318812
1   chromedriver                        0x00000001006d2d00 chromedriver + 4287744
2   chromedriver                        0x00000001003047ec chromedriver + 296940
3   chromedriver                        0x00000001002fd67c chromedriver + 267900
4   chromedriver                        0x00000001002ec9a4 chromedriver + 199076
5   chromedriver                        0x00000001002edef8 chromedriver + 204536
6   chromedriver                        0x00000001002eccd0 chromedriver + 199888
7   chromedriver                        0x00000001002ebb48 chromedriver + 195400
8   chromedriver                        0x00000001002ebaf4 chromedriver + 195316
9   chromedriver                        0x00000001002e9fa4 chromedriver + 18

An error occurred while visiting http://gwichin.strengthwebsitehosting.com/gwichinCommunitiesInuvikNativeBand: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: headless chrome=117.0.5938.149)
Stacktrace:
0   chromedriver                        0x0000000104e3265c chromedriver + 4318812
1   chromedriver                        0x0000000104e2ad00 chromedriver + 4287744
2   chromedriver                        0x0000000104a5c7ec chromedriver + 296940
3   chromedriver                        0x0000000104a5567c chromedriver + 267900
4   chromedriver                        0x0000000104a449a4 chromedriver + 199076
5   chromedriver                        0x0000000104a45ef8 chromedriver + 204536
6   chromedriver                        0x0000000104a44cd0 chromedriver + 199888
7   chromedriver                        0x0000000104a43b48 chromedriver + 195400
8   chromedriver                        0x0000000104a43af4 chromedriver + 195316
9   chromedriver                        0x0000

In [22]:
# Save the DataFrame to a new Excel file
df.to_excel('bandinfo_emails_data.xlsx', index=False)